<a href="https://colab.research.google.com/github/SURYAKNIGHT17/DATA_SCIENTIST/blob/main/Ip_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socket

def dns_resolver():
    domain = input("Enter domain name: ")

    try:
        # Perform DNS lookup
        ip_address = socket.gethostbyname(domain)
        print(f"IP Address for {domain} is: {ip_address}")
    except socket.gaierror:
        # Handle DNS resolution failure
        print(f"Unable to resolve host: {domain}")
    except Exception as e:
        # Handle any other exceptions
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    dns_resolver()


Enter domain name: www.google.com
IP Address for www.google.com is: 172.217.214.105


In [ ]:
!pip install folium pillow
import socket
import requests
import folium
import json
import ipaddress
from PIL import Image
from io import BytesIO
from IPython.display import display, Image as IPyImage

def dns_resolver(domain):
    try:
        # Perform DNS lookup
        ip_address = socket.gethostbyname(domain)
        print(f"\nDomain: {domain}")
        print(f"IP Address for {domain} is: {ip_address}")

        # Check if IP is public or private
        if is_public_ip(ip_address):
            print("IP Type: Public")
        else:
            print("IP Type: Private")

        # Reverse DNS lookup
        reverse_domain = reverse_dns_lookup(ip_address)
        if reverse_domain:
            print(f"Reverse DNS Lookup Result: {reverse_domain}")

        # Get geolocation data for the IP address
        geo_data = get_ip_geolocation(ip_address)
        if geo_data:
            display_location_details(geo_data)
            display_asn_info(geo_data)

            # Display interactive map with location marker
            latitude = geo_data.get("lat")
            longitude = geo_data.get("lon")
            display_map(latitude, longitude)

            # Display website logo
            display_website_logo(domain)

            # Option to export the data to a file
            export_to_file(geo_data, f"{domain}_dns_info.json")
        else:
            print("Geolocation data not available.")

    except socket.gaierror:
        print(f"Unable to resolve host: {domain}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

def reverse_dns_lookup(ip_address):
    try:
        host = socket.gethostbyaddr(ip_address)
        return host[0]  # Returns the primary domain name
    except socket.herror:
        return None

def get_ip_geolocation(ip_address):
    url = f"http://ip-api.com/json/{ip_address}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data["status"] == "success":
            return data
    except requests.RequestException as e:
        print(f"Error fetching geolocation data: {e}")
    return None

def display_asn_info(geo_data):
    asn = geo_data.get("as")
    isp = geo_data.get("isp")
    org = geo_data.get("org")
    country = geo_data.get("country")
    print(f"ASN: {asn}, ISP: {isp}, Organization: {org}, Country: {country}")

def display_location_details(geo_data):
    region = geo_data.get("regionName")
    city = geo_data.get("city")
    timezone = geo_data.get("timezone")
    postal_code = geo_data.get("zip")
    latitude = geo_data.get("lat")
    longitude = geo_data.get("lon")
    print(f"Location Details - Region: {region}, City: {city}, Timezone: {timezone}, Postal Code: {postal_code}")
    print(f"Coordinates - Latitude: {latitude}, Longitude: {longitude}")

def display_map(latitude, longitude):
    # Create a folium map centered at the specified latitude and longitude
    map_folium = folium.Map(location=[latitude, longitude], zoom_start=12)
    # Add a marker for the location
    folium.Marker([latitude, longitude], popup="Location").add_to(map_folium)
    # Display the map
    display(map_folium)

def display_website_logo(domain):
    logo_url = f"https://www.google.com/s2/favicons?sz=64&domain={domain}"
    try:
        response = requests.get(logo_url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        img.show()  # Show logo in a new window
        display(IPyImage(data=BytesIO(response.content).getvalue(), format='png'))
        print("Displayed website logo.")
    except Exception as e:
        print(f"Error displaying website logo: {e}")

def export_to_file(data, filename):
    with open(filename, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Information saved to {filename}")

def is_public_ip(ip_address):
    ip = ipaddress.ip_address(ip_address)
    return ip.is_global

def dns_batch_resolver():
    domains = input("Enter domain names separated by commas: ").split(",")
    for domain in domains:
        domain = domain.strip()
        if domain:
            dns_resolver(domain)

if __name__ == "__main__":
    dns_batch_resolver()
